In [1]:
import torch
import numpy as np
from seed import set_seed
from data_generator import gendata_Linear
from data_generator import gendata_Deep
set_seed(114)
test_data = gendata_Deep(500, 0.5)
set_seed(1145)
val_data = gendata_Deep(100, 0.5)
train_data = gendata_Deep(400, 0.5)

In [4]:
show_val = True
n_epoch = 100
n_lr = 1e-3
n_node = 50
n_layer = 2
patiences = 10

X_train = torch.tensor(train_data['X'], dtype=torch.float32)
A_train = torch.tensor(train_data['A'], dtype=torch.float32)
R_0_train = torch.tensor(train_data['R'], dtype=torch.float32)
logR_0_train = torch.log(R_0_train)
#A_tr = (A_train,1-A_train)

X_val = torch.tensor(val_data['X'], dtype=torch.float32)
A_val = torch.tensor(val_data['A'], dtype=torch.float32)
R_0_val = torch.tensor(val_data['R'], dtype=torch.float32)
logR_0_val = torch.log(R_0_val)

X_test = torch.tensor(test_data['X'], dtype=torch.float32)
A_test = torch.tensor(test_data['A'], dtype=torch.float32)
R_0_test = torch.tensor(test_data['R'], dtype=torch.float32)
logR_0_test = torch.log(R_0_test)
# t_nodes = torch.tensor(t_nodes, dtype=torch.float32)
# t_fig = torch.tensor(t_fig, dtype=torch.float32)
d_X = X_train.size()[1]


In [7]:
class DNNModel(torch.nn.Module):
    def __init__(self, in_features=d_X , out_features=3, hidden_nodes=n_node, hidden_layers=n_layer, drop_rate=0):
        super(DNNModel, self).__init__()
        layers = []
        # Input layer
        layers.append(torch.nn.Linear(in_features, hidden_nodes))
        layers.append(torch.nn.ReLU())
        layers.append(torch.nn.Dropout(drop_rate))
        # Hidden layers
        for _ in range(hidden_layers):
            layers.append(torch.nn.Linear(hidden_nodes, hidden_nodes))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(drop_rate))
        # Output layer
        layers.append(torch.nn.Linear(hidden_nodes, out_features))
        self.linear_relu_stack = torch.nn.Sequential(*layers)
    
    def forward(self, x):
        return self.linear_relu_stack(x)

# Initialize model and optimizer
model = DNNModel()
optimizer = torch.optim.Adam(model.parameters(), lr=n_lr)

# Custom loss function for binary decision: via MSEloss
# my_loss = torch.nn.MSELoss(reduction='mean')
def my_loss(mu_TX, A, logR):  #mu_TX is 3 dim, A = (1,A) is 3 dim, logR is 1 dim
    A1 = torch.ones(A.shape[0])
    A1 = A1.unsqueeze(1)
    A = torch.cat((A1,A),dim = 1) # add the baseline mu0
    mu_TX1 = torch.sum(mu_TX*A, dim = 1)
    loss_fun = (logR-mu_TX1)**2  #least square
    return loss_fun.mean()


    #simultaneously calculate the intrgral for A=0 and A=1 (need to be corrected)


# Treat and Untreat
A_untreat = torch.zeros(A_test.shape, dtype=torch.float32)
A_untreat[:,0]=1   #untreat
A_treated = torch.zeros(A_test.shape, dtype=torch.float32)
A_treated[:,1]=1   #treated


# Early stopping parameters
best_val_loss = float('inf')
patience_counter = 0


In [8]:

for epoch in range(n_epoch):
    model.train()  # Set model to training mode
        
    # mu_TX = model(torch.cat((logR_0_train.unsqueeze(1), X_train), dim=1)).squeeze()
    mu_TX = model(X_train).squeeze()
    # int_exp_g_TX = vectorized_gaussian_quadrature_integral(batch_func_train, torch.zeros_like(R_O_train), R_O_train, n_points=100,A=A_train) # batch_size = 0.8n

    loss = my_loss(mu_TX, A_train, logR_0_train)
    # print('epoch=', epoch, 'loss=', loss.detach().numpy())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation step
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        # g_TX_val = model(torch.cat((R_O_val.unsqueeze(1), X_val), dim=1)).squeeze()
        mu_TX_val = model(X_val).squeeze()
        # int_exp_g_TX_val = vectorized_gaussian_quadrature_integral(batch_func_val, torch.zeros_like(R_O_val), R_O_val, n_points=100,A=A_val) # batch_size = 0.2n
        
        # val_loss = my_loss(g_TX_val, int_exp_g_TX_val, A_val)
        val_loss = my_loss(mu_TX_val, A_val, logR_0_val)
        if show_val == True:
            print('epoch=', epoch, 'val_loss=', val_loss.detach().numpy())
    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict()  # Save the best model state
    else:
        patience_counter += 1
        if show_val == True:
            print('patience_counter =', patience_counter)
        
    if patience_counter >= patiences:
        if show_val == True:
            print(f'Early stopping at epoch {epoch + 1}, ', 'validation—loss=', val_loss.detach().numpy())
        break

# Restore best model if needed
model.load_state_dict(best_model_state)

epoch= 0 val_loss= 1.422686
epoch= 1 val_loss= 1.3716295
epoch= 2 val_loss= 1.3203996
epoch= 3 val_loss= 1.2691836
epoch= 4 val_loss= 1.2171057
epoch= 5 val_loss= 1.1645485
epoch= 6 val_loss= 1.1113552
epoch= 7 val_loss= 1.057157
epoch= 8 val_loss= 1.0019766
epoch= 9 val_loss= 0.94627136
epoch= 10 val_loss= 0.8898612
epoch= 11 val_loss= 0.83348274
epoch= 12 val_loss= 0.7776851
epoch= 13 val_loss= 0.7231549
epoch= 14 val_loss= 0.6708809
epoch= 15 val_loss= 0.6217648
epoch= 16 val_loss= 0.5775416
epoch= 17 val_loss= 0.5398294
epoch= 18 val_loss= 0.5103374
epoch= 19 val_loss= 0.49043941
epoch= 20 val_loss= 0.48062503
epoch= 21 val_loss= 0.4807231
patience_counter = 1
epoch= 22 val_loss= 0.48924598
patience_counter = 2
epoch= 23 val_loss= 0.503352
patience_counter = 3
epoch= 24 val_loss= 0.51891553
patience_counter = 4
epoch= 25 val_loss= 0.5316055
patience_counter = 5
epoch= 26 val_loss= 0.5382284
patience_counter = 6
epoch= 27 val_loss= 0.5373431
patience_counter = 7
epoch= 28 val_loss= 

<All keys matched successfully>

In [14]:
model.eval()
mu_TX_test = model(X_test).squeeze()
# print(mu_TX_test)
mu_base_test = mu_TX_test[:,0]
mu_un_test = mu_TX_test[:,1]
mu_tr_test = mu_TX_test[:,2]
At1 = torch.ones(A_test.shape[0])
At1 = At1.unsqueeze(1)
A_untreat = torch.zeros(A_test.shape, dtype=torch.float32)
A_untreat[:,0]=1   #untreat
A_treated = torch.zeros(A_test.shape, dtype=torch.float32)
A_treated[:,1]=1   #treated
A_untreat = torch.cat((At1,A_untreat),dim=1)
A_treated = torch.cat((At1,A_treated),dim=1)
Cmean_untreat = torch.exp(torch.sum(A_untreat*mu_TX_test,dim=1))
Cmean_treated = torch.exp(torch.sum(A_treated*mu_TX_test,dim=1))
print(mu_TX_test)

tensor([[0.9379, 0.2890, 0.4402],
        [0.8536, 0.2639, 0.6871],
        [0.6677, 0.2777, 0.4412],
        ...,
        [0.9023, 0.3251, 0.4768],
        [0.7182, 0.2660, 0.4716],
        [0.9605, 0.3063, 0.7504]], grad_fn=<SqueezeBackward0>)


In [15]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import pinv
import scipy.optimize as spo
from data_generator import gendata_Linear
from seed import set_seed
set_seed(114)
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
test_data = gendata_Linear(500, 0.5,beta,gamma)

set_seed(1194)
val_data = gendata_Linear(400, 0.5,beta,gamma)
train_data = gendata_Linear(1600, 0.5,beta,gamma)


In [7]:
from log_linear_binary import E_ols_bin
resl = E_ols_bin(train_data, val_data, test_data)
print(resl['intercept'])
print((resl['beta'])-beta)
print((resl['gamma'])-gamma)
Cmean_R_X_treated = resl['Cmean_R_X_treated']
Cmean_R_X_untreat = resl['Cmean_R_X_untreat']

0.9999874137376902
[-4.10426417e-05 -3.26473845e-04  1.14387192e-04  7.97832423e-04
  2.06182755e-04]
[-7.26571972e-05  6.27652062e-04 -6.25337711e-05 -4.22235312e-04
 -4.13108551e-04]


In [8]:
from optITR import optITR_lin_bin
itr = optITR_lin_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
print(itr['mis_rate'])

0.0


In [22]:
import numpy as np
# from numpy.linalg import inv
# from numpy.linalg import pinv
# import scipy.optimize as spo
from data_generator import gendata_Linear
from seed import set_seed
from log_linear_binary import E_lasso_bin
from optITR import optITR_lin_bin
set_seed(114)
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
test_data = gendata_Linear(500, 0.5,beta,gamma)

set_seed(1194)
val_data = gendata_Linear(100, 0.5,beta,gamma)
train_data = gendata_Linear(400, 0.5,beta,gamma)

reslasso = E_lasso_bin(train_data, val_data, test_data)
print(reslasso['intercept'])
print((reslasso['beta'])-beta)
print((reslasso['gamma'])-gamma)
Cmean_R_X_treated = reslasso['Cmean_R_X_treated']
Cmean_R_X_untreat = reslasso['Cmean_R_X_untreat']
itr = optITR_lin_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
print(itr['mis_rate'])

0.9999376071690771
[-0.          0.00089461 -0.00082154  0.00075583 -0.00078075]
[-0.00069768  0.00070852 -0.         -0.00062871  0.00067701]
0.0
